<div style='float: right'><img src='pic/kakkuro.png'/></div>

## <div id='kakkuro' />カックロ

In [ ]:
#!pip3 install pulp ortoolpy
import re, numpy as np, pandas as pd
from pulp import LpProblem, lpDot, lpSum, value
from ortoolpy import addvar, addvars, addbinvar, addbinvars
data = """\
#..##
....#
..#..
#....
##..#""".splitlines()
hint_v = [ # 開始行、開始列、個数、合計
    (0,1,4,11),
    (0,2,2,4),
    (1,0,2,14),
    (1,3,4,10),
    (2,4,2,3),
    (3,2,2,3),
]
hint_h = [ # 開始行、開始列、個数、合計
    (0,1,2,5),
    (1,0,4,17),
    (2,0,2,6),
    (2,3,2,4),
    (3,1,4,10),
    (4,2,2,3),
]

### 問題
* マスに 1～9 の数字を入れます
* 縦または横に連続する空マスの中に同じ数字は入りません
* 連続するマスの合計が示されます

### 定式化
\begin{array}{cl}
            変数 & v_{ijk} \in \{0, 1\} ~ \forall i, j, k ~ ~ ~ マスi,jが数字k+1か (1) \\
                 & r_{ij} \in Z ~ \forall i, j ~ ~ ~ ~ ~ マスi,jの数字 (2) \\
\mbox{subject to} & \sum_k{v_{ijk}} = 1 ~ \forall i, j ~ ~ ~ ~ 数字は1つ (3) \\
                 & \sum_k{k \times v_{ijk}} + 1 = r_{ij} ~ \forall i, j ~ ~ ~ rをvで表現 (4) \\
                 & \sum_{ij}{r_{ij}} = 合計 ~ \forall i, j ~ ~ ~ ~ ~ マスの合計が同じ (5) \\
                 & マスの並びで同じ数字は禁止 (6) \\
\end{array}

In [ ]:
ni,nj = len(data),len(data[0])
a = pd.DataFrame([(i,j,k) for i in range(ni) for j in range(nj)
    if data[i][j]=='.' for k in range(1,10)], columns=list('行列数'))
a['Var'] = addbinvars(len(a))
a[:2]

In [ ]:
m = LpProblem()
for _,v in a.groupby(['行','列']):
    m += lpSum(v.Var) == 1 # (1)
for (di,dj),hint in zip([(1,0),(0,1)],[hint_v,hint_h]):
    for si,sj,nl,sm in hint:
        b = a.query(f'{si}<=行<={si+nl*di}&{sj}<=列<={sj+nl*dj}')[['数','Var']]
        for _,v in b.groupby('数'):
            m += lpSum(v.Var) <= 1 # (2)
        m += lpDot(*b.T.values) == sm # (3)
m.solve()

In [ ]:
a['Val'] = a.Var.apply(value)
r = a[a.Val>0.5].数[::-1].tolist()
print(re.sub('\\.', lambda _: str(r.pop()), '\n'.join(data)))